In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import random
import ipyplot
import math

from utils import *

load_dotenv()
Migrator().run()

In [ ]:
query = "volume1"
models = Model.find(Model.filename % query).all()

In [ ]:
triton_client = grpcclient.InferenceServerClient(url=TRITON_SERVER_URL, verbose=False)

model_metadata = triton_client.get_model_metadata(
    model_name=FPENET_MODEL_NAME, model_version=MODEL_VERSION
)

model_config = triton_client.get_model_config(
    model_name=FPENET_MODEL_NAME, model_version=MODEL_VERSION
).config

input_names = [i.name for i in model_config.input]
output_names = [i.name for i in model_config.output]

In [ ]:
def render_fpenet_image(clip, points):
    img = Image.fromarray(
        clip.squeeze().transpose().transpose().astype("uint8"), "L"
    ).convert("RGB")
    left_pupils = points[68:71]
    right_pupils = points[72:75]
    arc_tans = []
    for p1, p2 in zip(left_pupils, right_pupils):
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        arc_tans.append(math.atan2(dy, dx))
    rotatation = (sum(arc_tans) / len(arc_tans)) * 100

    return img.rotate(rotatation)

In [ ]:
n = 0
max_images = 8
show_me_grid = []
show_me_labels = []
results = []
for model in tqdm(models):
    try:
        if (model.faces) and (n <= max_images):
            try:
                results.append(
                    submit_to_fpenet(
                        model,
                        input_names[0],
                        output_names,
                        request_id=model.filename,
                        timeit=False,
                        return_clips=True,
                    )
                )
                n += 1
            except Exception as e:
                print("There was an exception: {}".format(e))
    except:
        pass

for responses, clips in results:
    for response, clip in zip(responses, clips):
        points = response.as_numpy(output_names[1]).squeeze()
        img = render_fpenet_image(clip, points)
        show_me_grid.append(img)
        show_me_labels.append(response.get_response().id)
ipyplot.plot_images(
    images=show_me_grid,
    img_width=160,
    labels=show_me_labels,
    show_url=False,
    max_images=max_images,
)

In [ ]:
for i in results[0][0]:
    print(i.as_numpy(output_names[1]).squeeze().shape)

In [ ]:
pbar = tqdm(
    total=len(models),
    desc="Submitting photos to {} at {}".format(FPENET_MODEL_NAME, TRITON_SERVER_URL),
)

results = []

with ThreadPoolExecutor() as executor:
    for chunk in chunked(models, THREAD_CHUNKS):
        futures = []
        for model in chunk:
            pbar.update()
            if model.faces:
                futures.append(
                    executor.submit(
                        submit_to_fpenet,
                        model,
                        input_names[0],
                        output_names,
                        request_id=model.pk,
                    )
                )

        for future in as_completed(futures):
            try:
                infer_results = future.result()
                for i, infer_result in enumerate(infer_results):
                    model = Model.get(infer_result.get_response().id)
                    image_points = (
                        infer_result.as_numpy(output_names[1]).squeeze().tobytes()
                    )
                    model.faces[i].descriptors = str(image_points)
                    model.save()

            except Exception as e:
                print(
                    "There was an exception: {}\n{}.".format(
                        e, infer_result.get_response().id
                    )
                )

In [ ]:
models = Model.find(Model.filename % query).all()

for model in random.sample(models, len(models) - 1):
    try:
        if len(model.faces) > 1:
            print(
                "Looks like {} has {} faces.\n".format(model.filename, len(model.faces))
            )
            break
        else:
            continue
    except:
        pass
print(model.dict())

___